In [2]:
#!/usr/bin/env python
__author__="Vitali Molchan"

import sys

from bs4 import BeautifulSoup
from string import find,strip

import glob
import time
import dateutil

import re
import pandas as pd
import numpy as np

import matplotlib.pylab as plt
import seaborn as sns; sns.set()
import difflib

from dbfread import DBF
from simpledbf import Dbf5

In [3]:
% matplotlib inline
from IPython.display import display, HTML

### Statistical data from CBS sources
> CBS (Central Bureau of Statistics) collect different kinds of statistical information. We are going to use the data about every gemeente & buurt (part of gemeente). The full description of the data can be found in https://www.cbs.nl/-/media/_pdf/2016/35/shape%202016%20versie%2010.zip. Data include the information about different economical and social parameters (average income, average propery price, population structure per age, etnic group, etc). 

In [4]:
cbsBuurt2015 = Dbf5("CBS20160831/" + "buurt_2015.dbf").to_dataframe()

In [5]:
cbsBuurt2015.head()

,BU_CODE,BU_NAAM,WK_CODE,GM_CODE,GM_NAAM,IND_WBI,WATER,POSTCODE,DEK_PERC,OAD,...,AV20_BIOS,AF_SAUNA,AF_ZONBNK,AF_ATTRAC,AV10ATTRAC,AV20ATTRAC,AV50ATTRAC,OPP_TOT,OPP_LAND,OPP_WATER
0,BU00030000,Appingedam-Centrum,WK000300,GM0003,Appingedam,1,NEE,9901,1,1161,...,1.0,21.6,21.9,17.4,0.0,1.0,11.0,90,84,5
1,BU00030001,Appingedam-West,WK000300,GM0003,Appingedam,1,NEE,9903,5,881,...,1.0,22.4,22.2,17.8,0.0,1.0,11.7,163,158,5
2,BU00030002,Appingedam-Oost,WK000300,GM0003,Appingedam,1,NEE,9902,1,1084,...,1.0,21.1,21.7,16.9,0.0,1.0,11.0,295,284,11
3,BU00030007,Verspreide huizen Damsterdiep en Eemskanaal,WK000300,GM0003,Appingedam,1,NEE,9903,4,348,...,1.0,22.1,21.7,17.7,0.0,1.0,11.6,559,540,19
4,BU00030008,Verspreide huizen ten zuiden van Eemskanaal,WK000300,GM0003,Appingedam,1,NEE,9902,1,72,...,1.1,19.6,19.5,15.0,0.0,1.5,13.5,582,554,28


In [6]:
# convert numerical attributes to float values and replace -99999999 with np.NaN
text_attributes = ['BU_CODE','BU_NAAM','WK_CODE','GM_CODE','GM_NAAM','WATER']
numeric_attributes = sorted(list(set(cbsBuurt2015.columns).difference(text_attributes)))
cbsBuurt2015 = pd.concat([cbsBuurt2015[text_attributes].astype('unicode'),cbsBuurt2015[numeric_attributes].astype(float).replace(-99999999, np.nan)],axis=1)
cbsBuurt2015.head()

,BU_CODE,BU_NAAM,WK_CODE,GM_CODE,GM_NAAM,WATER,AANTAL_HH,AANT_INW,AANT_MAN,AANT_VROUW,...,P_VERWEDUW,P_WEST_AL,P_WONT2000,P_WONV2000,STED,STERFT_TOT,WONINGEN,WOZ,WWB_UITTOT,WW_UIT_TOT
0,BU00030000,Appingedam-Centrum,WK000300,GM0003,Appingedam,NEE,1290.0,2330.0,1085.0,1240.0,...,13.0,6.0,84.0,16.0,3.0,40.0,1459.0,121.0,80.0,70.0
1,BU00030001,Appingedam-West,WK000300,GM0003,Appingedam,NEE,1340.0,3115.0,1535.0,1580.0,...,4.0,5.0,92.0,8.0,4.0,25.0,1328.0,164.0,60.0,80.0
2,BU00030002,Appingedam-Oost,WK000300,GM0003,Appingedam,NEE,2700.0,5960.0,2865.0,3095.0,...,8.0,9.0,85.0,15.0,3.0,95.0,2833.0,120.0,320.0,200.0
3,BU00030007,Verspreide huizen Damsterdiep en Eemskanaal,WK000300,GM0003,Appingedam,NEE,120.0,340.0,180.0,160.0,...,2.0,6.0,46.0,54.0,5.0,0.0,123.0,244.0,0.0,10.0
4,BU00030008,Verspreide huizen ten zuiden van Eemskanaal,WK000300,GM0003,Appingedam,NEE,40.0,105.0,55.0,45.0,...,2.0,1.0,95.0,5.0,5.0,0.0,42.0,NaN,0.0,0.0


In [12]:
#delete all rows with empty postcode
cbsBuurt2015.dropna(subset=['POSTCODE'],inplace=True)
cbsBuurt2015['POSTCODE'] = cbsBuurt2015['POSTCODE'].astype(np.int)

In [10]:
print "Number of Buurts in CBS data is {}".format(cbsBuurt2015.shape[0])
print "Number of attributes for each Buurt is {}".format(cbsBuurt2015.shape[1])

Number of Buurts in CBS data is 12184
Number of attributes for each Buurt is 127


In [13]:
# Check if every buurt can be uniquely identified by Buurt name, Gemeente name and Postcode
cbsBuurt2015.groupby(['GM_NAAM','BU_NAAM','POSTCODE']).size().sort_values(ascending=False)

GM_NAAM             BU_NAAM                              POSTCODE
Oldebroek           Verspreide huizen Oosterwolde        8097        2
Staphorst           Verspreide huizen Lankhorst          7951        2
Goeree-Overflakkee  Achthuizen (gedeeltelijk)            3256        2
Noordoostpolder     Kraggenburg-bedrijventerrein         8317        2
Ermelo              Verspreide huizen bosgebied          3852        2
Ferwerderadiel      Janum                                9107        1
                    Hogebeintum                          9173        1
                    Hallum                               9074        1
                    Genum                                9174        1
                    Ferwerd                              9172        1
                    Blija                                9171        1
                    Birdaard                             9112        1
Etten-Leur          Schoenmakershoek                     4871        1
           

There are five Buurts which have the same postcode.To handle thess cases several options can be considered:
 - Calculate some sort of average for each attribute
 - Take the attributes values from the fist / last match
 <br /> Let's take for simplicity the attributes values from the first match:

In [14]:
cbsBuurt2015 = cbsBuurt2015.groupby(['GM_NAAM','BU_NAAM','POSTCODE']).first()
cbsBuurt2015.reset_index(inplace=True)
print cbsBuurt2015.shape

(12179, 127)


In [15]:
cbsBuurt2015.head()

,GM_NAAM,BU_NAAM,POSTCODE,BU_CODE,WK_CODE,GM_CODE,WATER,AANTAL_HH,AANT_INW,AANT_MAN,...,P_VERWEDUW,P_WEST_AL,P_WONT2000,P_WONV2000,STED,STERFT_TOT,WONINGEN,WOZ,WWB_UITTOT,WW_UIT_TOT
0,'s-Gravenhage,Archipelbuurt,2585,BU05180546,WK051805,GM0518,NEE,3210.0,5720.0,2700.0,...,4.0,30.0,94.0,6.0,1.0,35.0,3591.0,376.0,60.0,90.0
1,'s-Gravenhage,Arendsdorp,2596,BU05180478,WK051804,GM0518,NEE,555.0,1310.0,625.0,...,9.0,29.0,100.0,0.0,1.0,35.0,866.0,483.0,10.0,20.0
2,'s-Gravenhage,Belgisch Park,2587,BU05180271,WK051802,GM0518,NEE,4130.0,7955.0,3935.0,...,5.0,23.0,99.0,1.0,2.0,110.0,4091.0,294.0,120.0,130.0
3,'s-Gravenhage,Bezuidenhout-Midden,2595,BU05182665,WK051826,GM0518,NEE,2120.0,3970.0,2000.0,...,3.0,27.0,80.0,20.0,1.0,20.0,2069.0,244.0,30.0,80.0
4,'s-Gravenhage,Bezuidenhout-Oost,2593,BU05182666,WK051826,GM0518,NEE,5175.0,8940.0,4315.0,...,3.0,22.0,98.0,2.0,1.0,45.0,5151.0,169.0,210.0,200.0


#### Load www.funda.nl data into DataFrame

Data were collected from <a href="http://www.funda.nl">www.funda.nl </a>
HTML parsing has been performed using BeautifulSoup library.

Funda database contains following attributes:
 - FILE_NAME - name of hte file on the server (used for data collection only)
 - PROPERTY_TYPE: house, appartment
 - PROPERTY_ID - unique identifier (used for data collection only)
 - POSTCODE_NUMBER - digital part of post code (e.g. 2274 from 2274AC)
 - POSTCODE_LETTERS - literal part of post code (e.g. AC from 2274AC)
 - GEMEENTE
 - BUURT
 - OFFER_DATE - date when has been placed on the site
 - SOLD_DATE - date when the property has been sold (according to the site)

In [284]:
funda = pd.read_csv("20160830/All_houses/20160830_database_clean.txt",sep=";",header=None,na_values='NA')
funda.columns = ['FILE_NAME','PROPERTY_TYPE','PROPERTY_ID','POSTCODE_NUMBERS','POSTCODE_LETTERS','GEMEENTE','BUURT','OFFER_DATE','SOLD_DATE','SOLD_PRICE','ASKING_PRICE','VVV_PRICE','YEAR_OF_BUILDING','PLOT_AREA','VOLUME','PLOT_SIZE','NROOMS','TOTAL_FLOORS','PROPERTY_FLOOR','INSULATION_TYPE','HEATING_TYPE','GARDEN_TYPE','GARDEN_AREA','PARKING_TYPE','ENERGY_LABEL','MAKLER_NAME','MAKLER_TEL','dummy'] 
number_of_lines = funda.shape[0]

In [281]:
data.head()

,FILE_NAME,PROPERTY_TYPE,PROPERTY_ID,POSTCODE_NUMBERS,POSTCODE_LETTERS,GEMEENTE,BUURT,OFFER_DATE,SOLD_DATE,SOLD_PRICE,...,PROPERTY_FLOOR,INSULATION_TYPE,HEATING_TYPE,GARDEN_TYPE,GARDEN_AREA,PARKING_TYPE,ENERGY_LABEL,MAKLER_NAME,MAKLER_TEL,dummy
0,appartement-4003351-boerhaavelaan-102-a-featur...,appartement,4003351,3112.0,LM,Schiedam,Rotterdamsedijk,15-1-2008,8-8-2015,129.000,...,Begane grond,Dubbel glas,Moederhaard,Achtertuin,50.0,Openbaar parkeren en betaald parkeren,C,Makelaardij Wassenaar,010-4737933,NaN
1,appartement-4043458-a-weg-10-25-features.html,appartement,4043458,9581.0,AK,Musselkanaal,Zuid,6-11-2007,14-3-2016,247.500,...,4e woonlaag,NaN,C.V.-ketel,NaN,NaN,NaN,B,Kral Westen Makelaardij B.V.,0599-581759,NaN
2,appartement-4047711-dingstraat-20-features.html,appartement,4047711,7101.0,GX,Winterswijk,Centrale deel,31-10-2007,11-11-2015,180.000,...,2e woonlaag,"Dakisolatie, muurisolatie, vloerisolatie en du...",C.V.-ketel,NaN,NaN,NaN,B,Enning Makelaardij b.v.,0543-516151,NaN
3,appartement-4096883-groningerstraat-72-d-featu...,appartement,4096883,9401.0,BM,Assen,Galgenveld,19-12-2007,4-1-2016,175.000,...,4e woonlaag,NaN,C.V.-ketel,NaN,NaN,NaN,A,Oosting Makelaars,0592-310144,NaN
4,appartement-4104014-weverstraat-11-features.html,appartement,4104014,7271.0,AJ,Borculo,Borculo Oude-Centrum,22-4-2008,21-9-2015,125.000,...,2e woonlaag,"Dakisolatie, muurisolatie, vloerisolatie en ge...",C.V.-ketel,NaN,NaN,NaN,C,Bijker makelaardij,0545-254444,NaN


In [291]:
funda.dropna(subset=['SOLD_PRICE'],inplace=True)
funda[(funda['POSTCODE_NUMBERS'].isnull())].head()



,FILE_NAME,PROPERTY_TYPE,PROPERTY_ID,POSTCODE_NUMBERS,POSTCODE_LETTERS,GEMEENTE,BUURT,OFFER_DATE,SOLD_DATE,SOLD_PRICE,...,PROPERTY_FLOOR,INSULATION_TYPE,HEATING_TYPE,GARDEN_TYPE,GARDEN_AREA,PARKING_TYPE,ENERGY_LABEL,MAKLER_NAME,MAKLER_TEL,dummy
160,appartement-47056149-de-vierhoek-7-features.html,appartement,47056149,NaN,NaN,NaN,Bebowde kom Marienvelde,24-2-2011,22-10-2015,149.000,...,Begane grond,NaN,C.V.-ketel en vloerverwarming geheel,Zonneterras,NaN,Openbaar parkeren,A,NaN,0544-375040,NaN
460,appartement-47226775-langs-de-rijn-bouwnummer-...,appartement,47226775,NaN,NaN,NaN,Groene Weide,24-11-2011,5-12-2015,249.500,...,NaN,NaN,NaN,NaN,NaN,NaN,A,Roel Willemsen Garantiemakelaars,026-3274455,NaN
1214,appartement-47936819-de-vierhoek-11-features.html,appartement,47936819,NaN,NaN,NaN,Bebowde kom Marienvelde,2-2-2011,18-3-2016,137.500,...,Begane grond,NaN,C.V.-ketel,Zonneterras,NaN,NaN,A,Gierkink Makelaars,0544377444,NaN
1300,appartement-48001382-galloway-bouwnr-97-featur...,appartement,48001382,NaN,NaN,NaN,Overige verspreide huizen,3-5-2014,13-10-2015,184.950,...,2e woonlaag,NaN,NaN,NaN,NaN,NaN,A,Van der Waaij Makelaars en Rentmeesters,035-5394470,NaN
1301,appartement-48001389-galloway-bouwnr-94-featur...,appartement,48001389,NaN,NaN,NaN,Overige verspreide huizen,3-5-2014,18-5-2015,179.500,...,2e woonlaag,NaN,NaN,NaN,NaN,NaN,A,Van der Waaij Makelaars en Rentmeesters,035-5394470,NaN


In [288]:
set(funda['POSTCODE_NUMBERS'].unique()).difference(cbsBuurt2015['POSTCODE'].unique())

{nan,
 1022.0,
 1026.0,
 1032.0,
 1059.0,
 1074.0,
 1081.0,
 1086.0,
 1104.0,
 1107.0,
 1352.0,
 2019.0,
 2172.0,
 2546.0,
 2561.0,
 2632.0,
 2693.0,
 2726.0,
 2981.0,
 3023.0,
 3036.0,
 3052.0,
 3069.0,
 3078.0,
 3191.0,
 3193.0,
 3642.0,
 3643.0,
 3962.0,
 4103.0,
 4302.0,
 4536.0,
 5122.0,
 5252.0,
 6370.0,
 6534.0,
 8254.0,
 8444.0,
 8532.0,
 8702.0}

In [16]:
cbsBuurt2015.to_pickle("Processed_data/cbsBuurt2015.pickle")